Wytłumaczenie kodu:
Dropdown: Pozwala użytkownikowi wybierać lokalizacje.
Mapa: Używam biblioteki dash_leaflet do stworzenia interaktywnej mapy.
Callback: Funkcja w Dash, która aktualizuje pozycję markera i centrum mapy na podstawie wybranej lokalizacji.
Po uruchomieniu tego kodu, Dash utworzy lokalny serwer, na którym będzie dostępna aplikacja z mapą i listą rozwijaną. Użytkownik będzie mógł wybrać lokalizację z listy, a mapa automatycznie skupi się na wybranej lokalizacji.

In [6]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_leaflet as dl

# Dane lokalizacji
locations = {
    'Baden_Baden': {'lat': 48.765640, 'lon': 8.228524},
    'Bozanska': {'lat': 54.473028, 'lon': 18.302179},
    'Walcz_Drugi': {'lat': 53.257039, 'lon': 16.521203}
}

# Inicjalizacja aplikacji Dash
app = dash.Dash(__name__)

# Struktura aplikacji
app.layout = html.Div([
    dl.Map([
        dl.TileLayer(),  # Warstwa kafelków mapy
        dl.Marker(position=(48.765640, 8.228524), id='map-marker')  # Marker z początkową pozycją
    ], id='map', style={'width': '100%', 'height': '50vh'}, center=(50, 10), zoom=4),
    html.Div([  # Kontener dla menu rozwijanego
        dcc.Dropdown(
            id='location-dropdown',
            options=[{'label': loc, 'value': loc} for loc in locations.keys()],
            value='Bozanska',  # Wartość domyślna
            style={'width': '40%'}  # Zmniejszenie szerokości dropdown
        )
    ], style={'position': 'absolute', 'bottom': 10, 'left': 10, 'width': '100%', 'z-index': '1000'})
], style={'position': 'relative'})

# Callback do aktualizacji markera i centrum mapy
@app.callback(
    [Output('map-marker', 'position'),
     Output('map', 'center')],
    [Input('location-dropdown', 'value')]
)
def update_map(selected_location):
    lat_lon = locations[selected_location]
    return (lat_lon['lat'], lat_lon['lon']), (lat_lon['lat'], lat_lon['lon'])

# Uruchomienie serwera
if __name__ == '__main__':
    app.run_server(debug=True)


w dashu1 beda pokazywane ogolne wykresy historyczne i beda dynamicznie sie zminiec na podstawie wybranej lokalizacji, tez obliczone wartosci dla paneli slonecznych

w dashu 2 bedzie to samo tylko z predykcja